<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/estimating_statevectors_from_circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Neural Networks to imitate Quantum Computers with Qiskit

Quantum computers are rare, and even simulating quantum computation is expensive. With neural networks you can simulate quantum computers. Qiskit provides many useful functions along the way.

## Data generation

To use neural networks, we need a large dataset from which to find patterns. Using Qiskit allows us to create this large dataset.

In [1]:
!pip install --quiet apache-beam==2.15.0 qiskit==0.13.0

     |████████████████████████████████| 2.9MB 10.9MB/s 
     |████████████████████████████████| 153kB 50.9MB/s 
     |████████████████████████████████| 1.2MB 55.2MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 1.6MB 39.7MB/s 
     |████████████████████████████████| 17.3MB 138kB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 983kB 55.6MB/s 
     |████████████████████████████████| 112kB 56.1MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 5.6MB 64.1MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 573kB 55.8MB/s 
     |████████████████████████████████| 21.7MB 111kB/s 
     

In [2]:
%%writefile generate_qc_dict.py
"""
Generates a dictionary describing a Quantum Circuit.
"""

import random

import numpy as np
import qiskit
from qiskit.circuit.random.utils import random_circuit


def qc_to_statevectors_dict(
  qc: qiskit.QuantumCircuit,
) -> dict:
  """
  Simulates a 5-qubit Quantum Circuit to generate
  its statevectors from 1024 simulated ideal runs of the 5-qubit Quantum Circuit.
  :param qc: The 5-qubit Quantum Circuit to simulate
  :return: The dictionary of statevectors generated from 1024 simulated ideal runs of the 5-qubit Quantum Circuit.
  """
  results = qiskit.execute(
    qc,
    qiskit.Aer.get_backend('qasm_simulator')
  ).result()
  counts = results.get_counts()

  q_n = 5
  keys = [str(bin(i + 1024))[-q_n:] for i in range(2 ** q_n)]
  my_counts = {"state_" + key: 0 for key in keys}

  for key in counts:
    my_counts["state_" + key] = counts[key]
  return my_counts


def type_of_gate(
  gate: qiskit.QuantumCircuit,
) -> str:
  """
  Gets the type of gate.
  """
  return str(type(gate[0])).split(".")[-1][:-2]


def gate_to_dict(
  gate
) -> dict:
  """
  Given a gate, returns one row of the CSV file describing the gate.

  >>> gate_to_json(None)
  {}

  >>> qc = get_unrolled_quantum_circuit_from_filename(
  >>>     filename="one_qasm_file/circuit_88.qasm"
  >>> )
  >>> for n, gate in enumerate(qc.data):
  >>>   print(gate)
  >>>   print(gate_to_json(gate))
  (<qiskit.extensions.standard.u3.U3Gate object at 0x7fb91ab6f5c0>, [Qubit(QuantumRegister(5, 'q'), 0)], [])
{"Gate_Type": "U3Gate", "Control": 0, "Target": -1, "Angle_1": 0.0, "Angle_2": 0.0, "Angle_3": 0.5}
  ...

  """

  # todo: do not return string in this function
  if gate is None:
    raise "gate is None"

  COL_QUANTITY = 6
  gate_type = type_of_gate(gate)
  gate_control = gate[1][0].index
  try:
    gate_target = gate[1][1].index
  except IndexError:
    gate_target = -1
  gate_angles = gate[0].params
  u3_gate_str_list = [gate_type, gate_control, gate_target]
  if gate_angles and len(gate_angles) == 3:
    adj_angle = lambda angle: float(angle / np.pi)
    u3_gate_str_list += list(map(adj_angle, gate_angles[0:2]))

  initial_length = len(u3_gate_str_list)
  for i in range(initial_length, COL_QUANTITY):
    u3_gate_str_list.append(-1)
  dict_json = {
    "Gate_Type": u3_gate_str_list[0],
    "Control": u3_gate_str_list[1],
    "Target": u3_gate_str_list[2],
    "Angle_1": u3_gate_str_list[3],
    "Angle_2": u3_gate_str_list[4],
    "Angle_3": u3_gate_str_list[5],
  }
  return dict_json


def qc_to_gates_dict(
  qc: qiskit.QuantumCircuit,
) -> dict:
  """
  Parses the gates of a Quantum Circuit
  into a common dictionary format
  that conforms to our JSON schema.
  :param qc: The Quantum Circuit to get the gates of.
  :return: The gates of the Quantum Circuit in a common dictionary format.
  For example: { "gate_00": {}, ..., "gate_40": {}}
  """
  n_gates = len(qc.data)
  MAX_N_GATES = 40

  NOT_EXIST_VALUE = -1
  # The dictionary that every blank gate uses
  BLANK_GATE_DICT = {
    "Gate_Number": NOT_EXIST_VALUE,
    "Gate_Type": "",
    "Control": NOT_EXIST_VALUE,
    "Target": NOT_EXIST_VALUE,
    "Angle_1": NOT_EXIST_VALUE,
    "Angle_2": NOT_EXIST_VALUE,
    "Angle_3": NOT_EXIST_VALUE,
  }
  n_digit_gate_number = len(str(MAX_N_GATES)) // 2

  gates_dict = {}

  if len(qc.data) > MAX_N_GATES:
    print("WARNING: More gates than capacity")
  for gate_number, gate in enumerate(qc.data):
    gate_dict = gate_to_dict(gate)
    gate_dict['Gate_Number'] = gate_number
    gate_dict_key = 'gate_' + str(gate_number).zfill(n_digit_gate_number)
    gates_dict[gate_dict_key] = gate_dict
  for gate_number in range(n_gates, MAX_N_GATES + 1):
    gate_dict = BLANK_GATE_DICT
    gate_dict_key = 'gate_' + str(gate_number).zfill(n_digit_gate_number)
    gates_dict[gate_dict_key] = gate_dict
  return gates_dict


def qc_to_qc_dict(
  qc: qiskit.QuantumCircuit,
) -> dict:
  """
  Converts the Quantum Circuit into a common dictionary format that conforms to our JSON schema.
  :param qc: The Quantum Circuit to convert into a common dictionary format.
  :return: The common dictionary format representation of the Quantum Circuit.
  """
  gates_dict = qc_to_gates_dict(qc=qc)
  statevectors_dict = qc_to_statevectors_dict(qc=qc)
  qc_dict = {
    "statevectors": statevectors_dict,
    "gates": gates_dict
  }
  return qc_dict


def generate_qc(

) -> qiskit.QuantumCircuit:
  """
  Generates a random Quantum Circuit.
  :return: The generated random Quantum Circuit.
  """
  min_depth = 1
  max_depth = 40
  depth = random.randint(min_depth, max_depth)
  qc = random_circuit(
    n_qubits=5,
    depth=depth,
    max_operands=2,
    measure=True
  )
  pass_ = qiskit.transpiler.passes.Unroller(['u3', 'cx'])
  pm = qiskit.transpiler.PassManager(pass_)

  # if the quantum circuit has more gates than allowed
  try:
    # check if the quantum circuit has no id gates
    qc2 = pm.run(qc)
  except qiskit.QiskitError:
    # Quantum Circuit had id gate, try again
    return generate_qc()

  # if the quantum circuit has more gates than allowed
  if len(qc2.data) > max_depth:
    # Quantum Circuit had too many gates, try again
    return generate_qc()

  # Quantum Circuit had allowed number and type of gates, therefore successful.
  return qc2


def generate_qc_dict(

) -> dict:
  """
  Generates a dictionary describing a random Quantum Circuit.
  :return: The generated dictionary describing a random Quantum Circuit.
  """
  qc = generate_qc()
  qc_dict = qc_to_qc_dict(
    qc=qc,
  )
  return qc_dict


Writing generate_qc_dict.py


In [3]:
"""
Locally generates a JSON describing Quantum Circuits.
"""

import apache_beam as beam

import generate_qc_dict


def main(

):
  """
  Runs Apache Beam pipeline to generate a JSON file describing
  random generated Quantum Circuits.
  :return: None
  """
  N_CIRCUITS_TO_GENERATE = 10
  # make a list with length equal to number of circuits
  #  we want to generate so we can map
  #  the filler element with an actual circuit

  with beam.Pipeline('DirectRunner') as pipeline:
    (pipeline
     | beam.Create(range(N_CIRCUITS_TO_GENERATE))
     | beam.Map(lambda element: generate_qc_dict.generate_qc_dict())
     | beam.io.WriteToText('random_quantum_circuits_as_bq_json')
     )

    pipeline.run()


/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [0]:
main()

In [5]:
!cat random_quantum_circuits_as_bq_json-00000-of-00001

{'statevectors': {'state_00000': 853, 'state_00001': 171, 'state_00010': 0, 'state_00011': 0, 'state_00100': 0, 'state_00101': 0, 'state_00110': 0, 'state_00111': 0, 'state_01000': 0, 'state_01001': 0, 'state_01010': 0, 'state_01011': 0, 'state_01100': 0, 'state_01101': 0, 'state_01110': 0, 'state_01111': 0, 'state_10000': 0, 'state_10001': 0, 'state_10010': 0, 'state_10011': 0, 'state_10100': 0, 'state_10101': 0, 'state_10110': 0, 'state_10111': 0, 'state_11000': 0, 'state_11001': 0, 'state_11010': 0, 'state_11011': 0, 'state_11100': 0, 'state_11101': 0, 'state_11110': 0, 'state_11111': 0}, 'gates': {'gate_0': {'Gate_Type': 'U3Gate', 'Control': 2, 'Target': -1, 'Angle_1': 0.0, 'Angle_2': 0.0, 'Angle_3': -1, 'Gate_Number': 0}, 'gate_1': {'Gate_Type': 'CnotGate', 'Control': 2, 'Target': 0, 'Angle_1': -1, 'Angle_2': -1, 'Angle_3': -1, 'Gate_Number': 1}, 'gate_2': {'Gate_Type': 'U3Gate', 'Control': 0, 'Target': -1, 'Angle_1': 0.0, 'Angle_2': 0.0, 'Angle_3': -1, 'Gate_Number': 2}, 'gate_3'

Now that you have the generated circuits, you can predict their statevectors.